In [1]:
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import precision_recall_fscore_support as score
from hyperopt import tpe, STATUS_OK, Trials, hp, fmin, STATUS_OK, space_eval
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import wandb
wandb.login()

wandb: Currently logged in as: betelegeuse. Use `wandb login --relogin` to force relogin


True

In [3]:
# 🐝 Step 2: Define sweep config

def sweep_config():
    sweep_configuration = {
        'method': 'bayes',
        'name': 'sweep',
        'metric': {'goal': 'maximize', 'name': 'val_auc'},
        'parameters':
        {
            'learning_rate': {'max': 0.3000, 'min': 0.0001},
            'n_estimators': {'max': 1000, 'min': 100},
            'max_depth': {'max': 10, 'min': 3},
            'gamma': {'max': 0.5, 'min': 0.0},
            'min_child_weight': {'max': 10.000, 'min': 0.000},
            'colsample_bytree': {'max': 1.000, 'min': 0.300},
            'reg_alpha': {'max': 0.001, 'min': 1e-05},
            'reg_lambda': {'max': 0.001, 'min': 1e-05}
         }
    }
    return sweep_configuration
sweep_configuration = sweep_config()

In [4]:
sweep_id = wandb.sweep(sweep=sweep_configuration, project='test')

Create sweep with ID: wnrcdgoq
Sweep URL: https://wandb.ai/betelegeuse/test/sweeps/wnrcdgoq


In [5]:
train_df = pd.read_csv("data/mitbih_train.csv", header=None)
test_df = pd.read_csv("data/mitbih_test.csv", header=None)
train_df.to_numpy()
test_df.to_numpy()

x = np.copy(train_df.loc[:,train_df.columns!=187].to_numpy())
y = np.copy(train_df.loc[:,train_df.columns==187].to_numpy())
y = y > 0
y = y.astype(int)

test_x = np.copy(test_df.loc[:,test_df.columns!=187].to_numpy())
test_y = np.copy(test_df.loc[:,test_df.columns==187].to_numpy())
test_y = test_y > 0
test_y = test_y.astype(int)

train_x, val_x, train_y, val_y = train_test_split(x, y, test_size = 0.33)

In [6]:
# Initiate XGBoost Classifier
xgboost = XGBClassifier(tree_method="gpu_hist",
                        predictor="gpu_predictor",
                        objective="binary:logistic",
                        verbosity=1)

# Print default setting
xgboost.get_params()

{'objective': 'binary:logistic',
 'use_label_encoder': False,
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'gamma': None,
 'gpu_id': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': None,
 'max_bin': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': None,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'predictor': 'gpu_predictor',
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': 'gpu_hist',
 'validate_parameters': None,
 'verbosity': 1}

In [7]:
best_score = 0
def objective():
    global train_x, train_y, best_score
    run = wandb.init(project='test')
    params = {
            'learning_rate': wandb.config.learning_rate,
            'n_estimators': wandb.config.n_estimators,
            'max_depth': wandb.config.max_depth,
            'gamma': wandb.config.gamma,
            'min_child_weight': wandb.config.min_child_weight,
            'colsample_bytree': wandb.config.colsample_bytree,
            'reg_alpha': wandb.config.reg_alpha,
            'reg_lambda': wandb.config.reg_lambda
    }
    model = XGBClassifier(tree_method="gpu_hist",
                          predictor="gpu_predictor",
                          objective="binary:logistic",
                          **params,
                          verbosity=1)
    model.fit(train_x,train_y, eval_metric="auc", eval_set=[(train_x, train_y), (val_x, val_y)], verbose=False, early_stopping_rounds=10)
    results = model.evals_result()
    pred=model.predict(test_x)
    f1_macro_score = f1_score(test_y, pred, average='macro')
    if f1_macro_score > best_score:
        best_score = f1_macro_score
        model.save_model(f'best_{round(best_score, 3)}.model')
    my_table = wandb.Table(params.update({'predict_f1_macro': f1_macro_score}))
    run.log({"Table Name": my_table})
    wandb.log({
        'train_auc': results["validation_0"]["auc"],
        'val_auc': results["validation_1"]["auc"],
        'predict_f1_macro': f1_macro_score
    })


In [8]:
wandb.agent(sweep_id, function=objective)

wandb: Agent Starting Run: 2yy820ud with config:
wandb: 	colsample_bytree: 0.3638633360189619
wandb: 	gamma: 0.2228763972948813
wandb: 	learning_rate: 0.1909228330589652
wandb: 	max_depth: 9
wandb: 	min_child_weight: 5.075840677946777
wandb: 	n_estimators: 595
wandb: 	reg_alpha: 0.0006597154351875349
wandb: 	reg_lambda: 0.000644294124023974
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


C:\Users\DELL\PycharmProjects\Final-Thesis\venv\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
C:\Users\DELL\PycharmProjects\Final-Thesis\venv\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


predict_f1_macro,▁
predict_f1_macro,0.96601


wandb: Agent Starting Run: fjio7uun with config:
wandb: 	colsample_bytree: 0.8140840958426081
wandb: 	gamma: 0.15842453656308852
wandb: 	learning_rate: 0.1285441494715439
wandb: 	max_depth: 5
wandb: 	min_child_weight: 2.321297313870614
wandb: 	n_estimators: 956
wandb: 	reg_alpha: 0.0005970959887268013
wandb: 	reg_lambda: 0.0007112359354882503
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


C:\Users\DELL\PycharmProjects\Final-Thesis\venv\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
C:\Users\DELL\PycharmProjects\Final-Thesis\venv\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


...

predict_f1_macro,▁
predict_f1_macro,0.96507


wandb: Agent Starting Run: xbv0sjge with config:
wandb: 	colsample_bytree: 0.6701584494021648
wandb: 	gamma: 0.3454374816557824
wandb: 	learning_rate: 0.2197170803022172
wandb: 	max_depth: 3
wandb: 	min_child_weight: 6.409356815816127
wandb: 	n_estimators: 625
wandb: 	reg_alpha: 0.0003615784154686869
wandb: 	reg_lambda: 0.0005620269520549776
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


C:\Users\DELL\PycharmProjects\Final-Thesis\venv\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
C:\Users\DELL\PycharmProjects\Final-Thesis\venv\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


predict_f1_macro,▁
predict_f1_macro,0.9541


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [9]:
# fig, ax = plt.subplots(figsize=(30, 30))
fig, ax = plt.subplots(figsize=(170, 170))
xgboost.plot_tree(my_model, num_trees=0, rankdir='LR',ax=ax)
plt.savefig("temp.pdf")

AttributeError: 'XGBClassifier' object has no attribute 'plot_tree'